
# ETL Foreign Exchange Rates from Bangko Sentral ng Pilipinas

## Todos:

1. Create a python script that will scrape exchange rate conversions from [Bangko Sentral ng Pilipinas](https://www.bsp.gov.ph/SitePages/Statistics/ExchangeRate.aspx) official website.
2. Save the raw unprocessed scraped data to `data/raw/rates` and name it as **rates_[timestamp].json**.
2. Use **pyspark** to do cleansing and transformation against the scraped data.
3. Save the processed data as **rates.json** to `data/processed/rates` folder.
4. Use `Postgres` db to store and analyze processed foreign exchange rates.
5. Schedule the ETL task to run on a daily basis.

## Prerequisites:

- **Selenium** is going to be used for automating browser operations.
- **WebDriver Manager (_optional_)** will provide us the driver we need to work with selenium.
- **BeautifulSoup** will parse the html markup to be generated by selenium.
- **PySpark** will be used for data wrangling, cleansing and transformation.

## Install required packages

You can install required packages by running the ff. command:

- Run from a terminal:

```
    pip install webdriver_manager bs4 selenium
```

- Run from a notebook:

```
    !pip install webdriver_manager bs4 selenium
```

## Creating the scraping tool using python

Start by importing all the necessary libraries

In [6]:
# Third-party libraries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Local imports
import json
import csv
from datetime import datetime

This one is optional, but if you want your browser to run only in the background, add the following lines of code: 

In [7]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True # This option will hide your browsers user interface

The following lines of code will do the ff:

1. Starts up a browser that will automatically redirect to **Bangko Sentral ng Pilipinas** official website.
2. Captures the website's page source and then stores it to `html` variable.
3. BeautifulSoup will then parse the value stored in `html`.
4. Uses the `.find()` method to search for the web element(s) which contains the data we needed to extract.
5. Saves the extracted data inside `data/raw` folder.

In [8]:
URL = "https://www.bsp.gov.ph/SitePages/Statistics/ExchangeRate.aspx"

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(URL)
driver.implicitly_wait(10)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

exchange_rate_table = soup.find(id="tb1")
table2 = soup.find(id="tb2")

headers = [
    "country",
    "unit",
    "symbol",
    "euro_equivalent",
    "us_dollar_equivalent",
    "phil_peso_equivalent"
]

row = []
rates = []

for index, _ in enumerate(exchange_rate_table.contents):
    row.clear()
    if index > 0:
        for index, tag in enumerate(_.contents):
            row.append(tag.text)
        rates.append(list(row))

# Appends row headers at the beginning of the list
rates.insert(0, headers) 

timestamp = datetime.now().strftime("%Y%m%d_%T").replace(":", "")

with open(f"data/raw/rates_{timestamp}.csv", "w", newline="\n") as f:
    writer = csv.writer(f)
    writer.writerows(rates)

C:\Users\john.delmundo\AppData\Local\Temp\ipykernel_27212\1261582902.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


## Create a spark session 

In [9]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("ETL Foreign Exchange Rates from Bangko Sentral ng Pilipinas") \
    .getOrCreate()

In [10]:
SparkConf().set("spark.app.name", "ETL Foreign Exchange Rates from Bangko Sentral ng Pilipinas")

In [11]:
spark.conf.get("spark.app.name")

'ETL Foreign Exchange Rates from Bangko Sentral ng Pilipinas'

In [12]:
spark.conf.get("spark.master")

'local[*]'

In [13]:
spark

## Define the structure of the schema

In here, we use the **StructType** and **StructField** python classes to define the structure of our schema. 

In [14]:
from pyspark.sql.types import (
    StructType, 
    StructField, 
    FloatType,
    StringType
)

path = "data/raw/rates_20230523_173600.csv"

struct = StructType([
    StructField("country", StringType(), False),
    StructField("unit", StringType(), False),
    StructField("symbol", StringType(), False),
    StructField("euro_equivalent", FloatType(), False),
    StructField("us_dollar_equivalent", FloatType(), False),
    StructField("phil_peso_equivalent", FloatType(), False)
])

df = spark \
    .read \
    .schema(struct) \
    .option("header", True) \
    .csv(path)

df.show()

+--------------------+--------+------+---------------+--------------------+--------------------+
|             country|    unit|symbol|euro_equivalent|us_dollar_equivalent|phil_peso_equivalent|
+--------------------+--------+------+---------------+--------------------+--------------------+
|     1 UNITED STATES|  DOLLAR|   USD|       0.924642|                 1.0|              55.797|
|             2 JAPAN|     YEN|   JPY|       0.006673|            0.007217|              0.4027|
|    3 UNITED KINGDOM|   POUND|   GBP|       1.149977|              1.2437|             69.3947|
|          4 HONGKONG|  DOLLAR|   HKD|        0.11812|            0.127747|              7.1279|
|       5 SWITZERLAND|   FRANC|   CHF|       1.030241|            1.114206|             62.1694|
|            6 CANADA|  DOLLAR|   CAD|       0.684971|            0.740796|             41.3342|
|         7 SINGAPORE|  DOLLAR|   SGD|       0.687006|            0.742997|              41.457|
|         8 AUSTRALIA|  DOLLAR

## Verify the schema

In [15]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- euro_equivalent: float (nullable = true)
 |-- us_dollar_equivalent: float (nullable = true)
 |-- phil_peso_equivalent: float (nullable = true)



## Viewing data summary

In [16]:
df.summary().show()

+-------+---------------+--------+------+------------------+--------------------+--------------------+
|summary|        country|    unit|symbol|   euro_equivalent|us_dollar_equivalent|phil_peso_equivalent|
+-------+---------------+--------+------+------------------+--------------------+--------------------+
|  count|             18|      18|    18|                17|                  17|                  17|
|   mean|           null|    null|  null|0.5888973548456252|  0.6368924719255815|   35.53669452802826|
| stddev|           null|    null|  null|0.6276651628587195|  0.6788198137459448|   37.87610591694862|
|    min|1 UNITED STATES|BAHT****|   AED|            6.2E-5|              6.7E-5|              0.0037|
|    25%|           null|    null|  null|           0.11812|            0.127747|              7.1279|
|    50%|           null|    null|  null|          0.614794|              0.6649|             37.0994|
|    75%|           null|    null|  null|          0.924642|             

## Transforming the data

In [17]:
from pyspark.sql.functions import split

df1 = df.select(
    split("country", " ", -1).alias("country"),
    "unit",
    "symbol",
    "euro_equivalent",
    "us_dollar_equivalent",
    "phil_peso_equivalent"
)

df1.select("country").show()

+--------------------+
|             country|
+--------------------+
| [1, UNITED, STATES]|
|          [2, JAPAN]|
|[3, UNITED, KINGDOM]|
|       [4, HONGKONG]|
|    [5, SWITZERLAND]|
|         [6, CANADA]|
|      [7, SINGAPORE]|
|      [8, AUSTRALIA]|
|        [9, BAHRAIN]|
|        [10, KUWAIT]|
| [11, SAUDI, ARABIA]|
|        [12, BRUNEI]|
|     [13, INDONESIA]|
|      [14, THAILAND]|
|[15, UNITED, ARAB...|
|[16, EUROPEAN, MO...|
|         [17, KOREA]|
|         [18, CHINA]|
+--------------------+



In [18]:
from pyspark.sql.functions import slice, concat_ws, size

df1 = df1.select(
    slice("country", 2, size("country")).alias("country"),
    "unit",
    "symbol",
    "euro_equivalent",
    "us_dollar_equivalent",
    "phil_peso_equivalent"
)

df1.select("country").show()

df1_view = df1.createOrReplaceTempView("exchange_rates")

+--------------------+
|             country|
+--------------------+
|    [UNITED, STATES]|
|             [JAPAN]|
|   [UNITED, KINGDOM]|
|          [HONGKONG]|
|       [SWITZERLAND]|
|            [CANADA]|
|         [SINGAPORE]|
|         [AUSTRALIA]|
|           [BAHRAIN]|
|            [KUWAIT]|
|     [SAUDI, ARABIA]|
|            [BRUNEI]|
|         [INDONESIA]|
|          [THAILAND]|
|[UNITED, ARAB, EM...|
|[EUROPEAN, MONETA...|
|             [KOREA]|
|             [CHINA]|
+--------------------+



In [19]:
from pyspark.sql.functions import concat_ws

df1 = df1.select(
    concat_ws(" ", "country").alias("country"),
    "unit",
    "symbol",
    "euro_equivalent",
    "us_dollar_equivalent",
    "phil_peso_equivalent"
)

df1.select("country").show()

df1_view = df1.createOrReplaceTempView("exchange_rates")

+--------------------+
|             country|
+--------------------+
|       UNITED STATES|
|               JAPAN|
|      UNITED KINGDOM|
|            HONGKONG|
|         SWITZERLAND|
|              CANADA|
|           SINGAPORE|
|           AUSTRALIA|
|             BAHRAIN|
|              KUWAIT|
|        SAUDI ARABIA|
|              BRUNEI|
|           INDONESIA|
|            THAILAND|
|UNITED ARAB EMIRATES|
|EUROPEAN MONETARY...|
|               KOREA|
|               CHINA|
+--------------------+



In [20]:
script = """SELECT 
                INITCAP(country) AS country, 
                unit, 
                symbol, 
                euro_equivalent, 
                us_dollar_equivalent,
                phil_peso_equivalent 
            from 
                exchange_rates
    """

res = spark.sql(script)
res.select("country").show()

+--------------------+
|             country|
+--------------------+
|       United States|
|               Japan|
|      United Kingdom|
|            Hongkong|
|         Switzerland|
|              Canada|
|           Singapore|
|           Australia|
|             Bahrain|
|              Kuwait|
|        Saudi Arabia|
|              Brunei|
|           Indonesia|
|            Thailand|
|United Arab Emirates|
|European Monetary...|
|               Korea|
|               China|
+--------------------+



In [21]:
from pyspark.sql.functions import (
    current_date, 
    current_timestamp
)

res.select("*").withColumn("timestamp", current_date()).show()
res = res.select("*").withColumn("timestamp", current_date())

+--------------------+--------+------+---------------+--------------------+--------------------+----------+
|             country|    unit|symbol|euro_equivalent|us_dollar_equivalent|phil_peso_equivalent| timestamp|
+--------------------+--------+------+---------------+--------------------+--------------------+----------+
|       United States|  DOLLAR|   USD|       0.924642|                 1.0|              55.797|2023-05-24|
|               Japan|     YEN|   JPY|       0.006673|            0.007217|              0.4027|2023-05-24|
|      United Kingdom|   POUND|   GBP|       1.149977|              1.2437|             69.3947|2023-05-24|
|            Hongkong|  DOLLAR|   HKD|        0.11812|            0.127747|              7.1279|2023-05-24|
|         Switzerland|   FRANC|   CHF|       1.030241|            1.114206|             62.1694|2023-05-24|
|              Canada|  DOLLAR|   CAD|       0.684971|            0.740796|             41.3342|2023-05-24|
|           Singapore|  DOLL

In [61]:
timestamp = datetime.now().strftime("%Y%m%d_%T").replace(":", "")

res.write \
    .mode("overwrite") \
    .format("csv") \
    .save(r"data\processed\rates\rates_{}".format(timestamp))